### <center>Universidade Federal do Rio Grande do Norte<br>Programa de Pós-Graduação em Engenharia Elétrica e Computação<br>Module: Data Science (Tópicos Especiais F)<br>Professor Ivanovitch Silva

### <center> Students: Marianne Diniz / Taline Nóbrega

## <center> Project 6: K-Nearest Neighbors<br><br>Pokémon - GO

## 1. Introduction

[Pokémon Go](https://pokemongolive.com/en/) is a free-to-play, location-based augmented reality game developed by [Niantic](https://en.wikipedia.org/wiki/Niantic_(company) that can be downloaded for iOS or Android. The game works by using phone's GPS for real-world location and augmented reality in order to bring up on the screen virtual creatures, called Pokémon. Basically, the main purpose is locate, capture, battle, and train the Pokémons.

## 2. Objectives

Using a dataset available at [Kaggle](https://www.kaggle.com/semioniy/predictemall/data) the project aims to analyze the probability of a type of Pokémon appears on a given location. The mathematical methodology consists of k-nearest neighbors

## 3. Dataset

As mentioned previously, the dataset was obtained from Kaggle and it consists of roughly 293,000 pokemon sightings. 
Besides the number of rows, the original data also presents a great number of columns. Each column comprehend a feature description.


- **pokemonId** - the identifier of a pokemon, should be deleted to not affect predictions. (numeric; ranges between 1 and 151)
- **latitude, longitude** - coordinates of a sighting (numeric)
- **appearedLocalTime** - exact time of a sighting in format yyyy-mm-dd'T'hh-mm-ss.ms'Z' (nominal)
- **cellId 90-5850m** - geographic position projected on a S2 Cell, with cell sizes ranging from 90 to 5850m (numeric)
- **appearedTimeOfDay** - time of the day of a sighting (night, evening, afternoon, morning)
- **appearedHour/appearedMinute** - local hour/minute of a sighting (numeric)
- **appearedDayOfWeek** - week day of a sighting (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday)
- **appearedDay/appearedMonth/appearedYear** - day/month/year of a sighting (numeric)
- **terrainType** - terrain where pokemon appeared described with help of GLCF Modis Land Cover (numeric)
- **closeToWater** - did pokemon appear close (100m or less) to water (Boolean, same source as above)
- **city** - the city of a sighting (nominal)
- **continent** (not always parsed right) - the continent of a sighting (nominal)
- **weather** - weather type during a sighting (Foggy Clear, PartlyCloudy, MostlyCloudy, Overcast, Rain, BreezyandOvercast, LightRain, Drizzle, BreezyandPartlyCloudy, HeavyRain, BreezyandMostlyCloudy, Breezy, Windy, WindyandFoggy, Humid, Dry, WindyandPartlyCloudy, DryandMostlyCloudy, DryandPartlyCloudy, DrizzleandBreezy, LightRainandBreezy, HumidandPartlyCloudy, HumidandOvercast, RainandWindy) // Source for all weather features
- **temperature** - temperature in celsius at the location of a sighting (numeric)
- **windSpeed** - speed of the wind in km/h at the location of a sighting (numeric)
- **windBearing** - wind direction (numeric)
- **pressure** - atmospheric pressure in bar at the location of a sighting (numeric)
- **weatherIcon** - a compact representation of the weather at the location of a sighting (fog, clear-night, partly-cloudy-night, partly-cloudy-day, cloudy, clear-day, rain, wind)
- **sunriseMinutesMidnight-sunsetMinutesBefore** - time of appearance relatively to sunrise/sunset Source
- **population density** - what is the population density per square km of a sighting (numeric, Source)
- **urban-rural** - how urban is location where pokemon appeared (Boolean, built on Population density, <200 for rural, >=200 and <400 for midUrban, >=400 and <800 for subUrban, >800 for urban)
- **gymDistanceKm**, pokestopDistanceKm - how far is the nearest gym/pokestop in km from a sighting? (numeric, extracted from this dataset)
- **gymIn100m-pokestopIn5000m** - is there a gym/pokestop in 100/200/etc meters? (Boolean)
- **cooc 1-cooc 151** - co-occurrence with any other pokemon (pokemon ids range between 1 and 151) within 100m distance - and within the last 24 hours (Boolean)
- **class** - says which pokemonId it is, to be predicted.

<br>
<br>
However, in order to achieve the purpose of this project, the dataset was filtered and just some columns were preserved; as can be seen in the following outputs.

In [102]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from math import radians, cos, sin, asin, sqrt

In [112]:
pokemon_data = pd.read_csv('pokemon_data.csv')

In [285]:
pokemon_data.head()

,pokemonId,latitude,longitude,appearedLocalTime,appearedTimeOfDay,appearedHour,appearedMinute,appearedDayOfWeek,appearedDay,appearedMonth,...,continent,weather,temperature,population_density,urban,suburban,midurban,rural,class,Distance
0,16,20.525745,-97.460829,2016-09-08T03:57:45,night,5,57,dummy_day,8,8,...,America,Foggy,25.5,2431.2341,True,True,True,False,Common,6273.408323
1,133,20.523695,-97.461167,2016-09-08T03:57:37,night,5,57,dummy_day,8,8,...,America,Foggy,25.5,2431.2341,True,True,True,False,Rare,6273.354532
2,16,38.903590,-77.199780,2016-09-08T03:57:25,night,5,57,dummy_day,8,8,...,America,Clear,24.2,761.8856,False,True,True,False,Common,5884.677578
3,13,47.665903,-122.312561,2016-09-08T03:56:22,night,5,56,dummy_day,8,8,...,America,PartlyCloudy,15.6,4842.1626,True,True,True,False,Common,9428.159469
4,133,47.666454,-122.311628,2016-09-08T03:56:08,night,5,56,dummy_day,8,8,...,America,PartlyCloudy,15.6,4842.1626,True,True,True,False,Rare,9428.107272


From the filtered dataset, named **pokemon_data**, it is possible to verify 23 columns, which are:

In [287]:
pokemon_data.columns

Index(['pokemonId', 'latitude', 'longitude', 'appearedLocalTime',
       'appearedTimeOfDay', 'appearedHour', 'appearedMinute',
       'appearedDayOfWeek', 'appearedDay', 'appearedMonth', 'appearedYear',
       'closeToWater', 'city', 'continent', 'weather', 'temperature',
       'population_density', 'urban', 'suburban', 'midurban', 'rural', 'class',
       'Distance'],
      dtype='object')

## 4. K-nearest neighbors

The k-nearest neighbors algorithm, or just kNN, is a non-parametric method used for classification and regression. The input consists of the k closest training examples in the feature space. The output depends on whether kNN is used for classification or regression. In this project we foccused on regression application, so the output is the property value for the object. This value is the average of the values of its k nearest neighbors. Using this metric we are able to predict the possibility of a Pokémon shows up based on others Pókemon.

So, the methodology can be explained by these few procedures:

- Find a few similar listings
- Calculate the avarage of the listings
- Set an avarage value as the common for the listings

The similarity metric applied was the distance, calculated using latitude and longitude parameters.

## 5. Haversine Formula

The K-nearest neighbors algorith implements the Euclidean distance, which expects numerical values. The features **latitude** and **longitude** are non-ordial values, because a smaller numerical value doesn't directly correspond to a smaller value in a meaningful way. Latitude and longitude value pairs describe a point on a geographic coordinate system. So, once we had chosen these features we needed to define another strategy - [Haversine Formula](https://en.wikipedia.org/wiki/Haversine_formula).

The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes (in radians).

## 6. The Code

After filtering the dataset and pointed out which similarity metric we were going to use, it was analyze the Pokémon classes. 
According to [**PokeGo.org**](http://www.pokego.org/rare-pokemon-list/) the Pokémons are divided by rarity level. Basicaly there are five groups: Common, Uncommon, Rare, Very Rare and Super Rare.
The list of all Pokémons was collect and for all rows of the dataset was attributed a class value according to the rarity level classification.

In [159]:
d = []

for i in range (0,len(pokemon_data)):
   if (pokemon_data.pokemonId.iloc[i] in (13,14,15,16,17,18,19,20,41,42)):
       c = 'Common'
       d.append(c)    
       
   elif (pokemon_data.pokemonId.iloc[i] in (10,11,12,21,22,23,24,29,30,31,32,33,34,43,44,45,46,47,48,49,
       58,59,60,61,62,69,70,71,84,85,92,93,94,96,97,98,99,120,121,127)):
       c = 'Uncommon'
       d.append(c)    
    
   elif (pokemon_data.pokemonId.iloc[i] in (88,89,106,107,108,113,129,130,137,142)):
       c = 'Very Rare'
       d.append(c)   
   elif (pokemon_data.pokemonId.iloc[i] in (83,132,144,145,146,150,151,115,122,131)):
       c = 'Super Rare'
       d.append(c)   
   else:
       c = 'Rare'
       d.append(c) 
    
pokemon_data['class'] = d

At this point, we have the datased with the class column correspondent to rarity level classification. 
The next step was defining the point of interest to measure the probability of occurrence of a specific Pokémon class. It was decided for Parque das Dunas location, which latitude and longitude values are described in the code below.

In [290]:
# Defining the latitude and longitude of an arbitrary location

#Parque das Dunas - Natal/RN
lat1 = -5.8352
lon1 = -35.1901

Using the latitude and longitude values of Parque das Dunas, it was calculated the haversine formula for all rows of the dataset. A new column was created in the dataset to receive these values. It was named **Distance**

In [291]:
def haversine(lat1, lon1, lat2, lon2):

    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2]) 

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers
    return c * r


In [292]:
distancia = []

for i in range (0,len(pokemon_data)):
    d = haversine (lat1, lon1, pokemon_data['latitude'].loc[i], pokemon_data['longitude'].loc[i])
    distancia.append(d)
    
pokemon_data['Distance'] = distancia

Now we have the distance for all Pokémons in the datased. With these values we are able to verify which Pokémon are closer and what is the class of them.

In [293]:
# Checking the 10 closer Pokémons
pokemon_data.sort_values(by='Distance', ascending=True).iloc[:10]

,pokemonId,latitude,longitude,appearedLocalTime,appearedTimeOfDay,appearedHour,appearedMinute,appearedDayOfWeek,appearedDay,appearedMonth,...,continent,weather,temperature,population_density,urban,suburban,midurban,rural,class,Distance
131527,116,-7.211944,-35.885708,2016-09-05T15:22:48,evening,17,22,Friday,5,8,...,America,PartlyCloudy,28.2,1754.2621,True,True,True,False,Rare,171.291622
247480,60,-12.257491,-38.961711,2016-09-03T16:51:14,evening,18,51,Wednesday,3,8,...,America,Clear,27.2,1754.2621,True,True,True,False,Uncommon,825.418712
247808,39,-12.258527,-38.961222,2016-09-03T16:43:46,evening,18,43,Wednesday,3,8,...,America,Clear,27.2,1754.2621,True,True,True,False,Rare,825.491134
247363,127,-12.258610,-38.962037,2016-09-03T16:56:50,evening,18,56,Wednesday,3,8,...,America,Clear,27.2,1754.2621,True,True,True,False,Uncommon,825.543941
247540,1,-12.259417,-38.962037,2016-09-03T16:49:57,evening,18,49,Wednesday,3,8,...,America,Clear,27.2,1754.2621,True,True,True,False,Rare,825.621316
200068,14,-12.985247,-38.444941,2016-09-04T14:24:55,evening,16,24,Thursday,4,8,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.440468
199589,16,-12.985776,-38.444037,2016-09-04T14:30:05,evening,16,30,Thursday,4,8,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.453441
199267,13,-12.986731,-38.444530,2016-09-04T14:34:05,evening,16,34,Thursday,4,8,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.572219
198807,42,-12.986008,-38.447406,2016-09-04T14:42:22,evening,16,42,Thursday,4,8,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.628147
79426,133,-12.986008,-38.447406,2016-09-06T09:42:22,morning,11,42,Saturday,6,8,...,America,Clear,24.6,3622.8896,True,True,True,False,Rare,871.628147


The quantity of closest Pokémon will be our **k** value for the k-nearest neighbors algorithm.

Our first analysis was with **k=20**

In [297]:
p = pokemon_data.sort_values(by='Distance', ascending=True).iloc[:20].reset_index()
p

,index,pokemonId,latitude,longitude,appearedLocalTime,appearedTimeOfDay,appearedHour,appearedMinute,appearedDayOfWeek,appearedDay,...,continent,weather,temperature,population_density,urban,suburban,midurban,rural,class,Distance
0,131527,116,-7.211944,-35.885708,2016-09-05T15:22:48,evening,17,22,Friday,5,...,America,PartlyCloudy,28.2,1754.2621,True,True,True,False,Rare,171.291622
1,247480,60,-12.257491,-38.961711,2016-09-03T16:51:14,evening,18,51,Wednesday,3,...,America,Clear,27.2,1754.2621,True,True,True,False,Uncommon,825.418712
2,247808,39,-12.258527,-38.961222,2016-09-03T16:43:46,evening,18,43,Wednesday,3,...,America,Clear,27.2,1754.2621,True,True,True,False,Rare,825.491134
3,247363,127,-12.258610,-38.962037,2016-09-03T16:56:50,evening,18,56,Wednesday,3,...,America,Clear,27.2,1754.2621,True,True,True,False,Uncommon,825.543941
4,247540,1,-12.259417,-38.962037,2016-09-03T16:49:57,evening,18,49,Wednesday,3,...,America,Clear,27.2,1754.2621,True,True,True,False,Rare,825.621316
5,200068,14,-12.985247,-38.444941,2016-09-04T14:24:55,evening,16,24,Thursday,4,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.440468
6,199589,16,-12.985776,-38.444037,2016-09-04T14:30:05,evening,16,30,Thursday,4,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.453441
7,199267,13,-12.986731,-38.444530,2016-09-04T14:34:05,evening,16,34,Thursday,4,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.572219
8,198807,42,-12.986008,-38.447406,2016-09-04T14:42:22,evening,16,42,Thursday,4,...,America,Clear,19.8,3622.8896,True,True,True,False,Common,871.628147
9,79426,133,-12.986008,-38.447406,2016-09-06T09:42:22,morning,11,42,Saturday,6,...,America,Clear,24.6,3622.8896,True,True,True,False,Rare,871.628147


In [311]:
p_dist = p.iloc[:20].pivot_table(index = ['class'],  values = 'Distance', aggfunc='count') 
p_dist

,Distance
class,
Common,8
Rare,7
Uncommon,5


In [312]:
# Predicting the probability of appearance for each class of Pokemon
# Predicting the probability of a pokemon of specific class appers at a given location.

k = 20
prob_common = (p_dist.iloc[0]/k)*100
prob_rare = (p_dist.iloc[1]/k)*100
prob_uncommon = (p_dist.iloc[2]/k)*100
prob_veryrare = (0/k)*100
prob_superrare = (0/k)*100

print('Predicted probability k = %.0f\n- Common: %.2f \n- Uncommon: %.2f\n- Rare: %.2f\n- Very Rare: %.2f\n- Super Rare: %.2f' % 
      (k,prob_common,prob_rare,prob_uncommon,prob_veryrare,prob_superrare))

Predicted probability k = 20
- Common: 40.00 
- Uncommon: 35.00
- Rare: 25.00
- Very Rare: 0.00
- Super Rare: 0.00


Now, considering **k = 500** to identify any differences.

In [317]:
p = pokemon_data.sort_values(by='Distance', ascending=True).iloc[:500].reset_index()
p_dist = p.iloc[:500].pivot_table(index = ['class'],  values = 'Distance', aggfunc='count') 

# Predicting the probability of appearance for each class of Pokemon
# Predicting the probability of a pokemon of specific class appers at a given location.

k = 500
prob_common = (p_dist.iloc[0]/k)*100
prob_rare = (p_dist.iloc[1]/k)*100
prob_uncommon = (p_dist.iloc[2]/k)*100
prob_veryrare = (p_dist.iloc[3]/k)*100
prob_superrare = (0/k)*100

print('Predicted probability k = %.0f\n- Common: %.2f \n- Uncommon: %.2f\n- Rare: %.2f\n- Very Rare: %.2f\n- Super Rare: %.2f' % 
      (k,prob_common,prob_rare,prob_uncommon,prob_veryrare,prob_superrare))

Predicted probability k = 500
- Common: 36.80 
- Uncommon: 25.40
- Rare: 32.40
- Very Rare: 5.40
- Super Rare: 0.00


## 7. Conclusion

Based on the results obtained by applying k-nearst neighbour, we can note that the possibility of a Super Rare Pokémon shows up in Natal/RN is almost null. The most common ones are the classes common and rare. 
Additionally, it is possible to verify that k-nearest neighbor algorithm works well to predict values. 